```

| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 하나 
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
                            .
                            .
                            .
                            .
                            .
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |  열차 백 

```

전체 구간에 대해 유전 알고리즘을 적용시킴니다.

In [133]:
# import 및 변수정의 구간
import random
import math
import re
import operator 
from random import randint

# 리스트 크기
listN = 8

# 염색체 해집단 수
N = 30

# 역 구간
Station_section = 11

# 하루에 운행하는 열차 수
One_day_train = 8

#교차율
Pc = 0.7

#변이율 역수 관계
Pm = 100

#마지막 세대수 
largest_generation = 5

#보존할 상위값
save_value = 5


In [134]:
# 01010로 이루어진 리스트를 10진수로 디코딩하는 함수

def decoding(X, Y=0):
    """
    input   : 0100로 이루어진 리스트
    return : 10진수 정수
    """
    for i, x in enumerate(X[1:]):
        Y += x* math.pow(2, listN -i -2)
    #10나누고 소스점 1자리 표시하는데 왜 
    #Y = round(Y/10, 1) 
    
    #처음 값이 0이면 음수로 return
    if X[0] == 0: return -Y
    return Y

In [135]:
decoding([0,1,1,0,0,1,1,1])

-103.0

In [136]:
#파일을 2차원 배열로 바꿔주기

def Readtxt(fname):
    """
    input  : 파일들
    return : 2차원 배열 
    """
    w = []
    with open(fname, 'r') as f :
        for line in f: #for each line in the file,
            w.append(line.split('\t'))
    return w

In [137]:
# 바뀌야됨
#def ScoreFunction(X, 구간,):
#                           몇번째 열차인지
def ScoreFunction(X,num_of_Train):
    """
    X에 속도 리스트를 넣어서 평가
    """
    #구간별 점수 (11개죠)
    sp = []
    for x in (X):
        sp.append(decoding(x))
        
    test = 0
    for i, _ in enumerate(sp):
        sp[i] = (777 - (6 * sp[i])) * 8
    
    return sp

In [138]:
test_X = ScoreFunction(speedlist[1][1],1)
test_X[1]

9672.0

In [139]:
#초기값
speedlist = [[[[random.choice([0,1]) for i in range(listN)]for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]



In [140]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]),len(speedlist[0][0][0]))

30 8 11 8


In [141]:
#하루 열차의 총합 구하기
def Make_Score_per_all(speedlist):
    #열차마다 점수                             #30개의 경쟁 100개의 열차
    Score_per_train = [[0 for i in range(One_day_train)]for j in range(N)]

    # 30개의 경쟁
    for k in range(N):
        # 100대의 열차
        for i in range(One_day_train):
            # 11개의 역
            one_tranin = ScoreFunction(speedlist[k][i], k)
            for j in range(Station_section):
                #각 열차에 대해      += 11개의 구간의 점수 합                   ([30경쟁][100대열차] +=  [30경쟁][100대열차][11개의 역])
                Score_per_train[k][i] += one_tranin[j]


    # 하루마다의 점수 
    Score_per_all = [0 for i in range(N)]

    # 30개의 경쟁            
    for i in range(N):
        # 100대의 열차
        for j in range(One_day_train):
             Score_per_all[i] += Score_per_train[i][j]
                
    return Score_per_all

In [142]:
Score_per_all = Make_Score_per_all(speedlist)
Score_per_all

[560784.0,
 523152.0,
 578352.0,
 529632.0,
 533520.0,
 532992.0,
 566544.0,
 519072.0,
 523440.0,
 508944.0,
 529680.0,
 566544.0,
 601728.0,
 606480.0,
 508992.0,
 555792.0,
 647424.0,
 550176.0,
 575904.0,
 559680.0,
 528720.0,
 552816.0,
 589824.0,
 505296.0,
 574272.0,
 509184.0,
 599760.0,
 525024.0,
 555744.0,
 591024.0]

In [143]:
print(len(speedlist), len(speedlist[0]), len(speedlist[0][0]), len(speedlist[0][0][0]),"총개수 : " ,len(speedlist) * len(speedlist[0]) * len(speedlist[0][0]) * len(speedlist[0][0][0])
)


30 8 11 8 총개수 :  21120


In [144]:
def Score_assessment_group(Score_per_all):
    # 적합도 비율 찾기
    speedlist_evaluation = [[Score_per_all[i], 0, i] for i in range(N)]

    Sum_of_fitness = 0
    for i in range(N): 
        Sum_of_fitness += Score_per_all[i]
    for i in range(N):
        if i == 0:
            speedlist_evaluation[i][1] = speedlist_evaluation[i][0]/Sum_of_fitness
        else:
            speedlist_evaluation[i][1] = speedlist_evaluation[i-1][1]  + speedlist_evaluation[i][0]/Sum_of_fitness
    
    return speedlist_evaluation


In [145]:
def roulettes(Group_assessment_value):
    roulette = random.random()
    for i in range(N):
        if i == 0:
            if 0 < roulette < Group_assessment_value[i][1]:
                break
        if Group_assessment_value[i-1][1] < roulette < Group_assessment_value[i][1]:
            break
    return i

In [146]:
#부모 변경
def Change_parent(speedlist, Selected_list_in_roulette):
    W = [[[0 for j in range(Station_section)]for k in range(One_day_train)]for z in range(N)]
    # 교차함수 및 변이함수 (자식만들기)
    for i in range(int(N/2)):
        for j in range(One_day_train):
            for k in range(Station_section):
                #교차
                selection = random.random()
                ###############################################################
                if selection < Pc:
                    rand = random.choice([i+1 for i in range(listN-1)]) # 0~ 이구간, 이구간 ~ 끝 교차 , 이구간 = 1~리스트 크기 -1 
                    # W(교차된 자식) = Selected_list_in_roulette(선택된 리스트에서) rand구간으로 나눈 값
                    W[i*2][j][k]     = speedlist[Selected_list_in_roulette[i*2]][j][k][0:rand]         + speedlist[Selected_list_in_roulette[i*2+1]][j][k][rand:]
                    W[i*2+1][j][k] = speedlist[Selected_list_in_roulette[i*2+1]][j][k][0:rand] + speedlist[Selected_list_in_roulette[i*2]][j][k][rand:]
                else:
                    W[i*2][j][k]     = speedlist[Selected_list_in_roulette[i*2]][j][k]
                    W[i*2+1][j][k] = speedlist[Selected_list_in_roulette[i*2+1]][j][k]
                #변의
                for z in range(2):
                    # Pm * 100 분의 1의 확률 
                    if randint(0, 100 * Pm) == 0:
                        #selection 부분을 변의 1 -> 0  0 -> 1
                        if W[i*2+z][j][k][rand] == 0:
                            W[i*2+z][j][k][rand] = 1
                        else:
                            W[i*2+z][j][k][rand] = 0
    return W

In [147]:
#세대 반복
def Evolution(speedlist, num_generation):
    while 1: 
        if num_generation >= largest_generation:
            break

        #전체 값으로 해서 
        Score_per_all = Make_Score_per_all(speedlist)

        #평가한 값 
        Group_assessment_value = Score_assessment_group(Score_per_all)

        #                                         정렬   점수값이 높은 순으로
        sortand_Group_assessmend = sorted(Group_assessment_value, key=operator.itemgetter(0),reverse=True)
        #상위 save_value 개값        
        save_assessmend_value = sortand_Group_assessmend[0:save_value]
        # Score_per_all에서 몇번 배열이 상위 5개인지만 
        # 리스트 해쉬값만
        save_assessmend_num = [save_assessmend_value[i][2] for i in range(save_value)]


        Selected_list_in_roulette = []

        #룰렛에 선택된 리스트들 마지막 5개는 상위5개만 
        for i in range(N-5):
            Selected_list_in_roulette.append(roulettes(Group_assessment_value))

        Selected_list_in_roulette += save_assessmend_num
        
        #룰렛 랜덤하게  근데이거하면 오류남 왜지?
        #Selected_list_in_roulette = random.shuffle(Selected_list_in_roulette)
        # ------------------------------------------------------------------------------------------------------------------------------------------------------------
        
        New_list = Change_parent(speedlist, Selected_list_in_roulette)
        
        New_Score_per_all = Make_Score_per_all(New_list)
        
        print("-" * 15)    
        print(num_generation,"번째를 합니다.")
        print("-" * 15)
        for i, nl in enumerate(New_Score_per_all):
            print(i, nl)
        
        speedlist = New_list
        num_generation += 1

    return New_list

In [148]:
test_list = []

test_list = Evolution(speedlist, 0)

---------------
0 번째를 합니다.
---------------
0 546576.0
1 544416.0
2 540288.0
3 643680.0
4 559728.0
5 518736.0
6 546144.0
7 554976.0
8 576864.0
9 584496.0
10 511920.0
11 584784.0
12 571632.0
13 543072.0
14 632448.0
15 519648.0
16 549936.0
17 539520.0
18 627504.0
19 516192.0
20 511152.0
21 543792.0
22 604368.0
23 498000.0
24 565776.0
25 643152.0
26 618720.0
27 597072.0
28 602640.0
29 584304.0
---------------
1 번째를 합니다.
---------------
0 555360.0
1 575424.0
2 497040.0
3 520608.0
4 593856.0
5 591072.0
6 588000.0
7 514944.0
8 511152.0
9 511152.0
10 513888.0
11 615744.0
12 621792.0
13 525168.0
14 570240.0
15 546912.0
16 568176.0
17 537264.0
18 549840.0
19 601296.0
20 528864.0
21 554304.0
22 536688.0
23 530304.0
24 554784.0
25 643824.0
26 631680.0
27 643920.0
28 625632.0
29 607248.0
---------------
2 번째를 합니다.
---------------
0 623136.0
1 539952.0
2 631680.0
3 584928.0
4 539856.0
5 548256.0
6 513360.0
7 616272.0
8 647760.0
9 497232.0
10 544704.0
11 548496.0
12 620736.0
13 541488.0
14 639408.0
1

In [149]:
test_list

[[[[0, 1, 1, 0, 1, 0, 1, 0],
   [0, 0, 1, 1, 0, 0, 1, 0],
   [0, 0, 0, 0, 1, 1, 1, 0],
   [0, 0, 1, 0, 1, 1, 0, 0],
   [0, 1, 1, 0, 0, 1, 0, 0],
   [0, 0, 1, 0, 0, 0, 1, 0],
   [1, 0, 0, 0, 0, 0, 1, 0],
   [0, 0, 1, 0, 0, 1, 0, 1],
   [1, 0, 0, 1, 1, 1, 0, 1],
   [0, 1, 0, 0, 0, 0, 0, 1],
   [0, 0, 0, 1, 1, 0, 1, 0]],
  [[0, 0, 1, 1, 1, 1, 0, 1],
   [1, 1, 1, 0, 0, 0, 1, 1],
   [0, 0, 1, 0, 0, 0, 1, 1],
   [1, 0, 0, 0, 0, 1, 1, 1],
   [1, 0, 1, 1, 1, 1, 1, 0],
   [1, 1, 0, 0, 0, 1, 0, 0],
   [1, 1, 1, 1, 0, 1, 0, 1],
   [1, 0, 1, 0, 1, 0, 0, 0],
   [0, 1, 0, 0, 1, 1, 0, 0],
   [0, 1, 1, 0, 0, 0, 1, 0],
   [0, 1, 1, 1, 0, 1, 1, 1]],
  [[0, 1, 1, 1, 1, 1, 0, 0],
   [0, 0, 1, 0, 1, 1, 1, 1],
   [1, 0, 0, 0, 1, 1, 0, 0],
   [0, 0, 0, 1, 0, 1, 0, 0],
   [0, 1, 0, 1, 1, 1, 0, 1],
   [1, 1, 1, 0, 1, 1, 0, 0],
   [0, 1, 0, 1, 1, 0, 0, 1],
   [0, 1, 0, 0, 1, 0, 1, 0],
   [1, 1, 1, 1, 1, 1, 0, 1],
   [0, 1, 1, 1, 0, 1, 1, 0],
   [1, 1, 0, 0, 0, 0, 0, 0]],
  [[1, 0, 1, 0, 0, 0, 1, 1],
   [1, 0, 1

In [150]:
if speedlist[1][1][1][1] == 0:
    print("!!")
else:
    print("??")

??


In [151]:
test_list2 = []

test_list2 = Evolution(test_list, 0)

---------------
0 번째를 합니다.
---------------
0 536928.0
1 520848.0
2 640560.0
3 619440.0
4 541680.0
5 646032.0
6 540480.0
7 591264.0
8 570768.0
9 657120.0
10 641952.0
11 633024.0
12 544896.0
13 647616.0
14 583200.0
15 641664.0
16 640176.0
17 646176.0
18 650448.0
19 641616.0
20 557472.0
21 632592.0
22 636336.0
23 636336.0
24 637632.0
25 654432.0
26 661536.0
27 634368.0
28 641664.0
29 652320.0
---------------
1 번째를 합니다.
---------------
0 632112.0
1 646512.0
2 641520.0
3 634800.0
4 652320.0
5 652320.0
6 648672.0
7 643440.0
8 651072.0
9 636624.0
10 627264.0
11 529776.0
12 637296.0
13 558000.0
14 628560.0
15 655392.0
16 544560.0
17 636144.0
18 531552.0
19 630384.0
20 580560.0
21 564048.0
22 614640.0
23 569952.0
24 627840.0
25 653136.0
26 661392.0
27 650160.0
28 643104.0
29 659664.0
---------------
2 번째를 합니다.
---------------
0 651120.0
1 626112.0
2 663696.0
3 651360.0
4 579648.0
5 621600.0
6 625728.0
7 630672.0
8 637392.0
9 646416.0
10 657360.0
11 636144.0
12 520176.0
13 643776.0
14 530400.0
1

In [152]:
Www = [[[0 for i in range(2)]for j in range(2)]for k in range(2)]
    # 교차함수 및 변이함수 (자식만들기)
    
print(Www)
for i in range(2):
    for j in range(2):
        for k in range(2):
            Www[i][j][k] = [43,43,0,2,2]
            
Www

[[[0, 0], [0, 0]], [[0, 0], [0, 0]]]


[[[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]],
 [[[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]],
  [[43, 43, 0, 2, 2], [43, 43, 0, 2, 2]]]]